# Naive Bayes

In [1]:
import os
import io
import re
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Retorna la ruta completa del archivo y su contenido (sin la cabecera)
def readFiles(path):
    #os.walk itera sobre todos los archivos de una carpeta
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                #se salta todo el encabezado y empieza a leer en el 
                #primera linea vacía
                elif line == '\n':
                    inBody = True
            f.close()
            #Join all items in a list into a string, using a \n as separator
            message = '\n'.join(lines)

            #Convierte a minusculas
            message = message.lower()
            #Eliminar URLs
            message = re.sub(r'http\S+|www\S+', '', message)
            #Elimina direcciones de correo
            message = re.sub(r'\S+@\S+', '', message)
            #Elimina numeros
            message = re.sub(r'\d+', '', message)
            #Elimina signos de puntuacion y caracteres especiales
            message = re.sub(r'[^a-z\s]', '', message)
            #Elimina espacios extra
            message = re.sub(r'\s+', ' ', message).strip()
            #Elimina etiquetas HTML (como <h1>, <input>, <div>, etc.)
            message = re.sub(r'<[^>]+>', ' ', message)
            #Elimina comentarios HTML
            message = re.sub(r'<!--.*?-->', ' ', message, flags=re.DOTALL)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

#Un DataFrame con dos columnas, uno contiene el contenido del correo y el otro el tipo (spam o ham) 
data = DataFrame({'message': [], 'class': []})

#data = data.append(dataFrameFromDirectory('./datos/emails/spam', 'spam'))
#data = data.append(dataFrameFromDirectory('./datos/emails/ham', 'ham'))

# Un DataFrame con dos columnas, uno contiene el contenido del correo y el otro el tipo (spam o ham)
data_spam = dataFrameFromDirectory('C:/Users/Gabriel/datos/emails/spam', 'spam')
data_ham = dataFrameFromDirectory('C:/Users/Gabriel/datos/emails/ham', 'ham')

data = pd.concat([data_spam, data_ham], ignore_index=True)

#Division en train (80%) y test (20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['class'])

# Ahora puedes usar:
# train_data['message'], train_data['class'] → para entrenar
# test_data['message'], test_data['class'] → para probar

print(f"Total: {len(data)}")
print(f"Train: {len(train_data)}")
print(f"Test: {len(test_data)}")


Total: 3000
Train: 2400
Test: 600


In [2]:
data.head()

,message,class
0,doctype html public wcdtd html transitionalen ...,spam
1,fight the risk of cancer slim down guaranteed ...,spam
2,fight the risk of cancer slim down guaranteed ...,spam
3,adult club offers free membership instant acce...,spam
4,i thought you might like these slim down guara...,spam


Usamos CountVectorizer para dividir cada mensaje en un listado de palabras. Luego lo introducimos en el clasificador MultinomialNB. Llamando a fit() obtenemos el filtro de spam entrenado.

In [3]:
vectorizer = CountVectorizer()
# Tokeniza todas las palabras, las convierte en números y luego las cuenta (cuantas veces aparece en cada mensaje individual)
counts = vectorizer.fit_transform(train_data['message'].values)

print('\ncounts')
print(type(counts))
counts
#Palabras y su frecuencia en cada documento
#print(counts)
#El vocabulario
#print(vectorizer.vocabulary_.items())


counts
<class 'scipy.sparse._csr.csr_matrix'>


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 294825 stored elements and shape (2400, 45385)>

In [4]:
targets = train_data['class'].values #Documentos con su clase

print('\ntargets')
print(type(targets))
print("Shape:", targets.shape)  
print("Dimensions:", targets.ndim)  
#print(targets)


targets
<class 'numpy.ndarray'>
Shape: (2400,)
Dimensions: 1


In [5]:
classifier = MultinomialNB()
classifier.fit(counts, targets)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [6]:
pred_train = classifier.predict(counts)
print("\n<<<>>> Resultados del modelo (TRAIN <<<>>>")
print(f"Exactitud (accuracy): {accuracy_score(targets, pred_train)*100:.2f}%") #Se calcula la puntuacion de clasificación de precisión.
print("\nMatriz de confusión (TRAIN):") # La matriz de confusion muestra los aciertos y errores separados por clase
#En scikit-learn, la matriz de confusión ordena las filas y columnas en orden alfabético por nombre de clase
print(confusion_matrix(targets, pred_train))
print("\nReporte de clasificación (TRAIN):") # El reporte de clasificacion muestra varias metricas para cada clase
print(classification_report(targets, pred_train))


<<<>>> Resultados del modelo (TRAIN <<<>>>
Exactitud (accuracy): 99.25%

Matriz de confusión (TRAIN):
[[2000    0]
 [  18  382]]

Reporte de clasificación (TRAIN):
              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      2000
        spam       1.00      0.95      0.98       400

    accuracy                           0.99      2400
   macro avg       1.00      0.98      0.99      2400
weighted avg       0.99      0.99      0.99      2400



Probamos con algunos ejemplos:

In [7]:
counts_test = vectorizer.transform(test_data['message'].values) # siempre se vectoriza
targets_test = test_data['class'].values
predictions = classifier.predict(counts_test)
predictions

array(['spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam',
       'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham',
       'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham',
 

In [8]:
print("\n<<<>> >Resultados del modelo (TEST) <<<>>>")
print(f"Exactitud (accuracy): {accuracy_score(targets_test, predictions)*100:.2f}%") #Se calcula la puntuacion de clasificación de precisión.
print("\nMatriz de confusión (TEST):")
print(confusion_matrix(targets_test, predictions)) # La matriz de confusion muestra los aciertos y errores separados por clase
#En scikit-learn, la matriz de confusión ordena las filas y columnas en orden alfabético por nombre de clase
print("\nReporte de clasificación (TEST):")
print(classification_report(targets_test, predictions)) # El reporte de clasificacion muestra varias metricas para cada clase



<<<>> >Resultados del modelo (TEST) <<<>>>
Exactitud (accuracy): 98.67%

Matriz de confusión (TEST):
[[500   0]
 [  8  92]]

Reporte de clasificación (TEST):
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       500
        spam       1.00      0.92      0.96       100

    accuracy                           0.99       600
   macro avg       0.99      0.96      0.98       600
weighted avg       0.99      0.99      0.99       600



500 → correos ham correctamente clasificados como ham (verdaderos negativos).

0 → correos ham mal clasificados como spam (falsos positivos).

8 → correos spam mal clasificados como ham (falsos negativos).

92 → correos spam correctamente clasificados como spam (verdaderos positivos).

Precision (precisión):
De todos los correos que el modelo dijo que eran spam, ¿cuántos realmente lo eran?

Recall (exhaustividad):
De todos los spam reales, ¿cuantos detecto el modelo?

F1-score:
Es un promedio entre precision y recall.
Cuanto más cerca de 1.0, mejor equilibrio entre ambos.

Support:
Cuántos ejemplos habia realmente de esa clase en el test.